<a href="https://colab.research.google.com/github/BustilloJuan/SparkR/blob/main/SparkR_Licorer%C3%ADa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Juan Fernández Bustillo

# En esta primera parte vamos a cargar las librerías necesarias y el dataset sobre el que vamos a trabajar.




In [1]:
!apt-get install openjdk-8-jdk-headless --quiet > /dev/null

In [2]:
!wget --quiet https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!rm spark-3.2.1-bin-hadoop3.2.tgz

In [3]:
%reload_ext rpy2.ipython

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [5]:
%%R
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))
sparkR.session(master = "local[*]")

R[write to console]: 
Attaching package: ‘SparkR’


R[write to console]: The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window


R[write to console]: The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union


R[write to console]: Spark package found in SPARK_HOME: /content/spark-3.2.1-bin-hadoop3.2



Launching java with spark-submit command /content/spark-3.2.1-bin-hadoop3.2/bin/spark-submit   sparkr-shell /tmp/Rtmp3s3zUb/backend_port48291be62a 
Java ref type org.apache.spark.sql.SparkSession id 1 


In [6]:
from google.colab import files
uploaded = files.upload()

Saving Iowa_Liquor_Sales5M_cleaned.csv to Iowa_Liquor_Sales5M_cleaned.csv


In [7]:
%%R
datos <- read.df(path="./Iowa_Liquor_Sales5M_cleaned.csv", source = "csv", delimiter=",", inferSchema = "true", header = "true", na.strings = "")

In [8]:
%%R
colnames(datos) #Visualizamos los nombres de las variables del dataset

 [1] "Invoice/Item Number"   "Date"                  "Store Number"         
 [4] "Store Name"            "Address"               "City"                 
 [7] "Zip Code"              "Store Location"        "County Number"        
[10] "County"                "Category"              "Category Name"        
[13] "Vendor Number"         "Vendor Name"           "Item Number"          
[16] "Item Description"      "Pack"                  "Bottle Volume (ml)"   
[19] "State Bottle Cost"     "State Bottle Retail"   "Bottles Sold"         
[22] "Sale (Dollars)"        "Volume Sold (Liters)"  "Volume Sold (Gallons)"


## Ejercicio 1:
Crear una columna llamada "Dollars per liter" que contenga cantidad de dólares que vale cada uno de los litros que se ha vendido. Para ello, se tiene que dividir el precio de venta
(Sale (Dollars)) entre los litros vendidos (Volume Sold (Liters)) por cada fila. Mostrar por pantalla
10 filas mostrando únicamente las columnas: "Invoice/Item Number" y " Dollars per liter".


In [9]:
%%R
NuevoDF <- withColumn(datos, "Dollars per liter", datos$"Sale (Dollars)"/datos$"Volume Sold (Liters)" ) #Creamos la nueva columna

In [10]:
%%R
head(select(NuevoDF, "Invoice/Item Number","Dollars per liter" ),10) #Mostramos los 10 primeros datos de esa nueva columna

   Invoice/Item Number Dollars per liter
1         S07596400023         16.986667
2         S13735200244         28.125000
3      INV-00047700020         17.994286
4      INV-11830200010         29.986667
5         S20150000021         15.680000
6      INV-11360800053         27.180000
7      INV-13465100013         19.920000
8      INV-14079600051         28.340000
9      INV-16352000017          5.931429
10        S13463000012         41.160000


## Ejericio 2:
 Se quiere conocer información sólo de las ventas de tequila y mezcal. Para ello, hay
que obtener un conjunto de datos que contenga por cada venta tres columnas: (a) la categoría
vendida (Category Name), que sólo puede ser tequila o mezcal, (2) la columna del nombre del
vendedor (Vendor Name) que hizo la venta, y (3) el precio de la venta (Sale (Dollars)). Mostrar
los primeros 10 primeros datos.

In [11]:
%%R
EJ2 <- select(datos, "Category Name", "Vendor Name", "Sale (Dollars)") #Nos quedamos solo con esas 3 columnas
nrow(EJ2) #Para ver cuantas filas tiene el dataset

[1] 17000


In [12]:
%%R
nombres <- unique(select(datos, "Category Name" )) #Para ver los diferentes niveles de la variable nombre de categoría
#collect(nombres) #Para mostrarlos

In [13]:
%%R
Filtrado <- filter(EJ2, EJ2$"Category Name" == "Mezcal" | EJ2$"Category Name" == "TEQUILA" ) #Filtramos aquellos que se correspondan con mezcal o tequila solo.
nrow(Filtrado) #Vemos sus filas para comprobar que hemos disminuido el tamaño.

[1] 509


In [14]:
%%R
head(Filtrado,10) #Simplemente mostramos los 10 primeros datos

   Category Name                  Vendor Name Sale (Dollars)
1        TEQUILA                      Proximo          17.25
2        TEQUILA              Jim Beam Brands          16.53
3        TEQUILA                      Proximo          15.74
4        TEQUILA        Sazerac North America         252.72
5        TEQUILA        Sazerac North America          59.64
6        TEQUILA               Luxco-St Louis         622.80
7        TEQUILA WILLIAM GRANT AND SONS, INC.          47.08
8        TEQUILA                      Proximo          46.38
9        TEQUILA   The Patron Spirits Company          12.75
10       TEQUILA                      Proximo          34.50


## Ejercicio 3:
Obtener todos los condados (County) que aparecen en los datos y para cada uno de
ellos obtener el número que tiene asociado (County Number). Posteriormente, ordenarlo de
mayor a menor número de condado (orden descendente). Notar que no debería haber ningún
condado duplicado, sino que se debe tener tantas filas como condados. Mostrar por pantalla 5
filas mostrando únicamente las columnas County y “County Number”. 


In [45]:
%%R
condados <- unique(select(datos, "County" )) #Todos los condados, que serían 217, algunos aparecen con número 
#collect(condados)  #Para mostrarlos, en este caso no lo hago, ya que son muchos

In [39]:
%%R
unicos<-unique(select(datos, "County","County Number"))

## En esta segunda visualización he visto que los condados representados por números tienen un número invalido "allamake", y se presenta un condado con <NA>, por ello voy a tratar de limpiar estos errores en los datos.

## He visto que había NA en los County, por ello elimino las filas que contengan NA de dicha columna. Al estar en valores únicos, logicamente es solo una fila.


In [50]:
%%R
SinNa <- dropna(datos, cols = "County") 
condados <- unique(select(SinNa, "County","County Number" ))
#collect(condados) #Una fila menos, por lo tanto lo ha realizado correctamente

In [91]:
%%R
limpios <- filter(condados, condados$ "County Number" > 0) #Como había algunos errores en el County Number, 
#ya que apraceían cómo ALLAMAKEE, he decidido filtrar por mayor de 0 para quedarme con unos datos limpios.
head(limpios)

     County County Number
1    Benton             6
2 ALLAMAKEE             3
3    Greene            37
4      Linn            57
5  Buchanan            10
6     Floyd            34


In [109]:
%%R
Minusculas <- select(limpios,lower(limpios$ "County"),limpios$"County Number") #Aquí lo que hago es pasa County a minúsculas
head(Minusculas)

  lower(County) County Number
1        benton             6
2     allamakee             3
3        greene            37
4          linn            57
5      buchanan            10
6         floyd            34


In [127]:
%%R
solos<-unique(select(Minusculas, "lower(County)", "County Number")) #Una vez pasado a minúsculas si hago el unique, quito los valores repetidos.
head(solos)

  lower(County) County Number
1         scott            82
2       osceola            72
3       carroll            14
4         adair             1
5    pocahontas            76
6          iowa            48


In [128]:
%%R
Ej3<-arrange(solos,desc(solos$ "County Number")) #Ordenados de mayor a menor.
head(Ej3,5)

  lower(County) County Number
1        wright            99
2         worth            98
3      woodbury            97
4    winneshiek            96
5     winnebago            95


## Ejercicio 4: 
¿Cuál es el vendedor (Vendor Name) que más litros (Volume Sold (Liters)) ha vendido en la ciudad (City) de Ottumwa? ¿Cuántos litros ha vendido ese vendedor en la ciudad?



In [137]:
%%R
EJ4 <- select(datos, "Vendor Name", "Volume Sold (Liters)", "City") #Selecciono las variables solicitadas 
head(EJ4)

         Vendor Name Volume Sold (Liters)       City
1    Diageo Americas                 3.00     DURANT
2    Diageo Americas                 1.12 BETTENDORF
3    Jim Beam Brands                 1.75  Knoxville
4  PERNOD RICARD USA                 1.50     Inwood
5    Jim Beam Brands                 0.50  DAVENPORT
6 Brown Forman Corp.                 0.50     Algona


In [145]:
%%R
Ottumwa <- filter(EJ4,EJ4$"City"=="Ottumwa") #Filtro por la ciudad pedida
head(Ottumwa)

               Vendor Name Volume Sold (Liters)    City
1       Heaven Hill Brands                  6.0 Ottumwa
2          Laird & Company                  1.5 Ottumwa
3 CONSTELLATION BRANDS INC                  9.0 Ottumwa
4          Jim Beam Brands                 12.0 Ottumwa
5        PERNOD RICARD USA                  0.4 Ottumwa
6          DIAGEO AMERICAS                  1.0 Ottumwa


In [150]:
%%R
Agregacion <- agg(groupBy(Ottumwa, Ottumwa$'Vendor Name'), "Total" = sum(Ottumwa$'Volume Sold (Liters)')) #Agregamos por la suma de las ventas
collect(Agregacion)

                        Vendor Name  Total
1  Constellation Wine Company, Inc.   9.60
2                   BACARDI USA INC   2.00
3                   Jim Beam Brands  13.75
4  Pernod Ricard USA/Austin Nichols   0.38
5                   Laird & Company  10.50
6                Brown Forman Corp.  17.10
7                 Phillips Beverage  30.00
8                         LUXCO INC  17.75
9                   DIAGEO AMERICAS 103.50
10                   Gemini Spirits   4.50
11            SAZERAC NORTH AMERICA  33.38
12         CONSTELLATION BRANDS INC  27.75
13               Heaven Hill Brands  25.50
14               E & J Gallo Winery  12.00
15             SAZERAC COMPANY  INC  36.53
16           REMY COINTREAU USA INC   1.50
17                PERNOD RICARD USA   5.65
18                   Luxco-St Louis  21.00
19                          PROXIMO  11.10


In [151]:
%%R
Ej4Final<-arrange(Agregacion,desc(Agregacion$ "Total")) #Ordenados de mayor a menor.
head(Ej4Final,1)

      Vendor Name Total
1 DIAGEO AMERICAS 103.5
